In [110]:
!rm -f *.py
!wget https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/dale_chall.py

--2021-11-22 18:03:01--  https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/dale_chall.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27456 (27K) [text/plain]
Saving to: ‘dale_chall.py’

dale_chall.py       100%[===================>]  26.81K  --.-KB/s    in 0.001s  

2021-11-22 18:03:01 (22.7 MB/s) - ‘dale_chall.py’ saved [27456/27456]



In [111]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [112]:
!rm -rf data*
!wget https://github.com/artificial-intelligence-ml-cti/ml_cti/raw/main/proiect/data.zip
!unzip data.zip

--2021-11-22 18:03:02--  https://github.com/artificial-intelligence-ml-cti/ml_cti/raw/main/proiect/data.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/data.zip [following]
--2021-11-22 18:03:02--  https://raw.githubusercontent.com/artificial-intelligence-ml-cti/ml_cti/main/proiect/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 741506 (724K) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>] 724.13K  --.-KB/s    in 0.03s   

2021-11-22 18:03:02 (24.0 MB/s) - ‘data.zip’ saved [741506/741506]


In [113]:
!echo "***\n lista fisiere :"
!ls data
!readlink -f data/

***\n lista fisiere :
test.xlsx  train.xlsx
/content/data


In [114]:
!pip install pyphen nltk pandas sklearn openpyxl

In [115]:
import numpy as np
import pandas as pd

from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score

from dale_chall import DALE_CHALL


In [116]:
dtypes = {"sentence":"string", "token" : "string", "complexity" : "float64"}
train = pd.read_excel('/content/data/train.xlsx', dtype = dtypes, keep_default_na = False)
test = pd.read_excel('/content/data/test.xlsx', dtype = dtypes, keep_default_na = False)

print('train.data: ', train.shape)
print('test.data: ', test.shape)

print(train.columns)

print(train['corpus'].unique())

print(len(train['token'].unique()))

print(len(train))

print(train.iloc[0])

train.data:  (7662, 4)
test.data:  (1338, 3)
Index(['corpus', 'sentence', 'token', 'complex'], dtype='object')
['bible' 'biomed' 'europarl']
3487
7662
corpus                                                  bible
sentence    Behold, there came up out of the river seven c...
token                                                   river
complex                                                     0
Name: 0, dtype: object


In [117]:

def is_dale_chall(word):
  return int(word in DALE_CHALL)
 
def is_title(word):
  return int(word.istitle())


def up_letters(word):
  nr = 0
  cnt=0
  for letter in word:
    cnt +=1
    if letter.isupper():
      nr += 1
  return nr/cnt

def litere_rare(word):
  litererare = 'jqxzJQXZ'
  nr = 0
  for letter in word:
    if letter in litererare:
      nr += 1
  return nr

def cifre(word):
  cifre = '0123456789'
  nr = 0
  for letter in word:
    if letter in word:
      return 1
    
  return 0

vowels = 'aeiouw'
def nr_vowels(word):
  nr = 0
  for chr in word:
    if chr in vowels:
      nr+=1
  return nr

def nr_synsets(word):
  return len(wn.synsets(word))/10

def get_word_structure_features(word):
  features = []
  features.append(is_dale_chall(word))
  features.append(is_title(word))
  features.append(up_letters(word))
  features.append(nr_vowels(word))
  features.append(cifre(word))
  features.append(litere_rare(word))
  return np.array(features)

def get_wordnet_features(word):
  features = []
  features.append(nr_synsets(word))
  return np.array(features)

def corpus_feature(corpus):
  d = {"bible" : [1,0,0], "europarl" : [0,1,0], "biomed" : [0,0,1]}
  return d[corpus]

print(get_word_structure_features('to'))


[1. 0. 0. 1. 1. 0.]


In [118]:
def featurize(row):
  word = row['token']
  all_features=[]
  all_features.extend(get_word_structure_features(word))
  all_features.extend(get_wordnet_features(word))
  all_features.extend(corpus_feature(row['corpus']))
  return np.array(all_features)

print(featurize(train.iloc[10]))

def featurize_df(df):
  nr_of_features = len(featurize(df.iloc[0]))
  nr_of_examples = len(df)
  features = np.zeros((nr_of_examples, nr_of_features))
  for index, row in df.iterrows():
    row_ftrs = featurize(row)
    features[index, :] = row_ftrs
  return features



[0.  0.  0.  1.  1.  0.  0.2 1.  0.  0. ]


ACURATETE NAIVE BAYES

In [119]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

xtrain, xtest, ytrain, ytest = train_test_split(x_train, y_train, test_size = 0.1, random_state=42)
gnb = GaussianNB()
predz = gnb.fit(xtrain,ytrain).predict(xtest)

print(balanced_accuracy_score(ytest,predz))

preds = gnb.fit(x_train,y_train).predict(x_test)
balanced_accuracy_score(ytest,predz)

0.7633282633282634


0.7633282633282634

In [120]:
df = pd.DataFrame()
df['id'] = test.index + len(train) + 1
df['complex'] = preds
df.to_csv('submission.csv', index=False)